<a href="https://colab.research.google.com/github/Surya-0804/Pytorch/blob/main/nut_and_bolts_of_dl_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
X=torch.randn(100,10)
y = torch.randint(0, 2, (100,))

In [ ]:
# Define the model
class TwoClassModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        logits = self.linear(x)
        probabilities = torch.softmax(logits, dim=1)
        return probabilities

In [ ]:
# Instantiate the model, loss function, and optimizer
model = TwoClassModel(10, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)

    # Backward and optimize
    optimizer.zero_grad()
    loss.retain_grad()
    loss.backward()
    print(loss.grad.item())
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [10/100], Loss: 0.6976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [20/100], Loss: 0.6950
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [30/100], Loss: 0.6925
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [40/100], Loss: 0.6901
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [50/100], Loss: 0.6877
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [60/100], Loss: 0.6855
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [70/100], Loss: 0.6834
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [80/100], Loss: 0.6814
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [90/100], Loss: 0.6795
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Epoch [100/100], Loss: 0.6778


In [ ]:
# Evaluate the model
with torch.no_grad():
    outputs = model(X)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y).float().mean()
    print(f'Accuracy: {accuracy.item()*100:.2f}%')

Accuracy: 58.00%


In [ ]:
#with hidden layers
from tqdm.notebook import tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the dataset
X = torch.randn(100, 20)  # 100 samples, 20 features
y = torch.randint(0, 10, (100,))  # 10 classes (0 to 9)

# Define the model
class TenClassModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        logits = self.fc3(x)
        probabilities = self.softmax(logits)
        return probabilities

# Instantiate the model, loss function, and optimizer
model = TenClassModel(20, 100, 100, 10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in tqdm(range(100)):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.grad.data = param.grad.data
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    outputs = model(X)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y).float().mean()
    print(f'Accuracy: {accuracy.item()*100:.2f}%')


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Loss: 2.2819
Epoch [20/100], Loss: 2.2444
Epoch [30/100], Loss: 2.1874
Epoch [40/100], Loss: 2.1193
Epoch [50/100], Loss: 2.0429
Epoch [60/100], Loss: 1.9685
Epoch [70/100], Loss: 1.9001
Epoch [80/100], Loss: 1.8325
Epoch [90/100], Loss: 1.7712
Epoch [100/100], Loss: 1.7325
Accuracy: 74.00%


**TorchVision**


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_dataset.data.shape
# x,y=train_dataset.data.next()

torch.Size([60000, 28, 28])

In [ ]:
# Define the model
class MNISTModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        logits = self.fc3(x)
        probabilities = torch.softmax(logits, dim=1)
        return probabilities

# Instantiate the model, loss function, and optimizer
model = MNISTModel(28 * 28, 400, 10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    # Forward pass
    outputs = model(train_dataset.data.float())
    loss = criterion(outputs, train_dataset.targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    outputs = model(test_dataset.data.float())
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == test_dataset.targets).float().mean()
    print(f'Accuracy: {accuracy.item()*100:.2f}%')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [2/10], Loss: 2.2558
Epoch [4/10], Loss: 2.1680
Epoch [6/10], Loss: 2.0829
Epoch [8/10], Loss: 1.9896
Epoch [10/10], Loss: 1.9157
Accuracy: 54.84%


In [ ]:
train_loader=torch.utils.data.DataLoader(train_dataset,64,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,64, shuffle=False)

In [ ]:
x,y=next(iter(train_loader))
x.shape,y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [ ]:
#using loaders
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    # Forward pass
    for batch_index,(data,targets) in enumerate(tqdm(train_loader)):
      outputs = model(data.float())
      loss = criterion(outputs, targets)

    # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (batch_index+1) % 100 == 0:
          print(f'Epoch [{epoch+1}/{num_epochs}],Batch [{batch_index+1}/{len(train_loader)}] Loss: {loss.item():.4f}')

# Evaluate the model
total=0
correct=0;
with torch.no_grad():
    for data,targets in test_loader:
      outputs = model(data.float())
      _, predicted = torch.max(outputs, 1)
      total+=targets.size(0)

      correct+= (predicted == targets).sum().item()
    print(f'Accuracy: {100*correct/total:.2f}%')


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch [1/10],Batch [100/938] Loss: 1.8598
Epoch [1/10],Batch [200/938] Loss: 1.7912
Epoch [1/10],Batch [300/938] Loss: 1.7378
Epoch [1/10],Batch [400/938] Loss: 1.6088
Epoch [1/10],Batch [500/938] Loss: 1.5984
Epoch [1/10],Batch [600/938] Loss: 1.6272
Epoch [1/10],Batch [700/938] Loss: 1.5762
Epoch [1/10],Batch [800/938] Loss: 1.5276
Epoch [1/10],Batch [900/938] Loss: 1.5223


  0%|          | 0/938 [00:00<?, ?it/s]

Epoch [2/10],Batch [100/938] Loss: 1.5351
Epoch [2/10],Batch [200/938] Loss: 1.4917
Epoch [2/10],Batch [300/938] Loss: 1.5269
Epoch [2/10],Batch [400/938] Loss: 1.4689
Epoch [2/10],Batch [500/938] Loss: 1.5351
Epoch [2/10],Batch [600/938] Loss: 1.5580
Epoch [2/10],Batch [700/938] Loss: 1.5026
Epoch [2/10],Batch [800/938] Loss: 1.5198
Epoch [2/10],Batch [900/938] Loss: 1.5226


...

Epoch [8/10],Batch [100/938] Loss: 1.4790
Epoch [8/10],Batch [200/938] Loss: 1.5075
Epoch [8/10],Batch [300/938] Loss: 1.4922
Epoch [8/10],Batch [400/938] Loss: 1.4768
Epoch [8/10],Batch [500/938] Loss: 1.5081
Epoch [8/10],Batch [600/938] Loss: 1.4925
Epoch [8/10],Batch [700/938] Loss: 1.4922
Epoch [8/10],Batch [800/938] Loss: 1.4956
Epoch [8/10],Batch [900/938] Loss: 1.4612


  0%|          | 0/938 [00:00<?, ?it/s]

Epoch [9/10],Batch [100/938] Loss: 1.4930
Epoch [9/10],Batch [200/938] Loss: 1.4920
Epoch [9/10],Batch [300/938] Loss: 1.4768
Epoch [9/10],Batch [400/938] Loss: 1.4612
Epoch [9/10],Batch [500/938] Loss: 1.5063
Epoch [9/10],Batch [600/938] Loss: 1.4912
Epoch [9/10],Batch [700/938] Loss: 1.5243
Epoch [9/10],Batch [800/938] Loss: 1.4924
Epoch [9/10],Batch [900/938] Loss: 1.4612


  0%|          | 0/938 [00:00<?, ?it/s]

Epoch [10/10],Batch [100/938] Loss: 1.4924
Epoch [10/10],Batch [200/938] Loss: 1.5148
Epoch [10/10],Batch [300/938] Loss: 1.4628
Epoch [10/10],Batch [400/938] Loss: 1.5117
Epoch [10/10],Batch [500/938] Loss: 1.4987
Epoch [10/10],Batch [600/938] Loss: 1.4924
Epoch [10/10],Batch [700/938] Loss: 1.4612
Epoch [10/10],Batch [800/938] Loss: 1.4993
Epoch [10/10],Batch [900/938] Loss: 1.4924
Accuracy: 96.23%


In [ ]:
import torch.nn.functional as F

class LeNet(nn.Module):
    def _init_(self):
        super(LeNet, self)._init_()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x